# RLS & CLS

In [0]:
%sql
-- Switch to the correct catalog for the project
USE CATALOG vstone_project;

-- Switch to the appropriate schema within the catalog
USE SCHEMA db_project;

# Building the Physical "Gold" Table

In [0]:
%sql
-- Create the physical table fact_car_sales with mapped city names based on car sale IDs
CREATE OR REPLACE TABLE vstone_project.db_project.fact_car_sales
USING DELTA
AS 
SELECT 
    id, 
    cost, 
    probeg, 
    currency, 
    -- Assign city_name based on id modulo 3 for demonstration purposes
    CASE 
      WHEN id % 3 = 0 THEN 'Кемерово'    -- Assign 'Кемерово' if id is divisible by 3
      WHEN id % 3 = 1 THEN 'Симферополь' -- Assign 'Симферополь' if remainder is 1
      ELSE 'Севастополь'                 -- Assign 'Севастополь' if remainder is 2
    END AS city_name, 
    load_timestamp 
FROM vstone_project.db_project.silver_car_sales;

# Creating the "Security Brain" (Functions)

In [0]:
%sql
-- Row-level security function: 
-- Managers see all rows; agents see only rows where city_name is 'Кемерово'
CREATE OR REPLACE FUNCTION vstone_project.db_project.row_filter_by_group(city_name_param STRING)
RETURN (
  IS_ACCOUNT_GROUP_MEMBER('vstone_managers') OR 
  (IS_ACCOUNT_GROUP_MEMBER('vstone_agents') AND city_name_param = 'Кемерово')
);

-- Column masking function for cost:
-- Managers see the real cost; all other users see 0.0
CREATE OR REPLACE FUNCTION vstone_project.db_project.cost_mask_by_group(cost_param DOUBLE)
RETURN (
  CASE 
    WHEN IS_ACCOUNT_GROUP_MEMBER('vstone_managers') THEN cost_param 
    ELSE 0.0 
  END
);

# Activating the "Shields" (ALTER TABLE)

In [0]:
%sql
-- Apply row-level security filter: Only managers see all rows; agents see rows where city_name = 'Кемерово'
ALTER TABLE vstone_project.db_project.fact_car_sales 
SET ROW FILTER vstone_project.db_project.row_filter_by_group ON (city_name);

-- Apply column-level masking: Only managers see actual cost; others see 0.0
ALTER TABLE vstone_project.db_project.fact_car_sales 
ALTER COLUMN cost SET MASK vstone_project.db_project.cost_mask_by_group;

-- Preview: Display sample rows with city_name, masked cost, currency, and load_timestamp
SELECT city_name, cost, currency, load_timestamp 
FROM vstone_project.db_project.fact_car_sales 
LIMIT 10;

# Verification & Audit

In [0]:
%sql
-- Show detailed metadata and properties for the fact_car_sales table
DESCRIBE TABLE EXTENDED vstone_project.db_project.fact_car_sales;